In [228]:
import pandas as pd
import numpy as np
import itertools
from datetime import datetime,timedelta,date
from collections import defaultdict



In [330]:
#df = pd.read_csv('../features/clear_data/rm_features.csv',';',infer_datetime_format=True,index_col=['time','host','consumer_group'],parse_dates=['time'])
#df = pd.read_csv('../features/clear_data/rm_features.csv',';',infer_datetime_format=True,index_col=['time'],parse_dates=['time'])
df = pd.read_csv('../features/clear_data/rm_features.csv',';',infer_datetime_format=True,parse_dates=['time'])

In [331]:
host_cons=[(host,group) for host,group in df.drop_duplicates(['host','consumer_group'])[['host','consumer_group']].values]


In [332]:
X_train=pd.DataFrame()
X_pred=pd.DataFrame()
for (host,consumer_group) in host_cons:
    sample=df[(df.host==host)&(df.consumer_group==consumer_group)]
    X_train=pd.concat([X_train,sample.head(int(len(sample)*0.97))],sort=False)
    X_pred=pd.concat([X_pred,sample[~sample.index.isin(X_train.index)]],sort=False)
#X_train=X_train.set_index(['time','host','consumer_group'])
#X_pred=X_pred.set_index(['time','host','consumer_group'])

In [225]:
# Разбить временной ряд на периоды с некоторым окном и свернуть каждый период с помощью PCA

#ts_period=1 # количество дней 
#n_length=24 # количество часов


#def window_stack(a, window=ts_period*n_length):
    # разбиваем данные на окна длиной ts_period
#    X=np.array([a[i:window+i] for i in range(int(len(a) - window))])
#    return X


#from sklearn.decomposition import PCA
#model_PCA = PCA(n_components=1)

#for (host,consumer_group) in host_cons:
#    samples=window_stack(df[(df.host==host)&(df.consumer_group==consumer_group)].set_index(['host','consumer_group']).values)
    #model_PCA.fit_transform(df)
#    for sample in samples:
        

In [297]:
#outliers_fraction = 0.0001
#def predict(model,X_train,X_pred):
#    model_predict=pd.DataFrame()
#    for (host,consumer_group) in host_cons:
#        sample_train=X_train[(X_train.host==host)&(X_train.consumer_group==consumer_group)].set_index(['time','host','consumer_group'])
#        sample_pred=X_pred[(X_pred.host==host)&(X_pred.consumer_group==consumer_group)].set_index(['time','host','consumer_group'])
#        model_predict=pd.concat([model_predict,pd.DataFrame(model.fit(sample_train).predict(sample_pred),columns=['is_anomaly'])],sort=False)
#    return model_predict.reset_index(drop=True)
        


In [336]:
outliers_fraction = 0.005
def predict(model,X_train,X_pred):
    return pd.DataFrame(model.fit(X_train.set_index(['time','host','consumer_group'])).predict(X_pred.set_index(['time','host','consumer_group'])),columns=['is_anomaly'])
        

## ML models

In [337]:
from sklearn.ensemble import IsolationForest

rns = np.random.RandomState(42)
model_IF = IsolationForest(random_state=rns,contamination=outliers_fraction,n_estimators =1000)

anomaly_IF = predict(model_IF,X_train,X_pred)



In [288]:
#from sklearn.neighbors import LocalOutlierFactor

#model_LOF = LocalOutlierFactor(n_neighbors=35, contamination=outliers_fraction)
#anomaly_LOF = predict(model_LOF,X_train,X_pred)

In [343]:
from sklearn import svm

model_SVM = svm.OneClassSVM(random_state=rns, nu=outliers_fraction, kernel="rbf", gamma=0.01)
anomaly_SVM = predict(model_SVM,X_train,X_pred)

In [341]:
from sklearn.covariance import EllipticEnvelope

model_EE = EllipticEnvelope(contamination=outliers_fraction)
anomaly_EE = predict(model_EE,X_train,X_pred)

C:\Users\JustMe\Anaconda3\lib\site-packages\sklearn\covariance\robust_covariance.py:622: UserWarning: The covariance matrix associated to your dataset is not full rank
  warnings.warn("The covariance matrix associated to your dataset "


In [193]:
#from sklearn.decomposition import PCA

#model_PCA = PCA(n_components=15, svd_solver='full')
#model_PCA.fit_transform(df)

#w_components_ = np.ones([model_PCA.n_components_, ])
#selected_components_ = model_PCA.components_[-1 * model_PCA.n_selected_components_:, :]
#selected_w_components_ = w_components_[-1 * model_PCA.n_selected_components_:]

PCA(copy=True, iterated_power='auto', n_components=15, random_state=None,
  svd_solver='full', tol=0.0, whiten=False)

In [338]:
df_predict=pd.concat([X_pred.reset_index(drop=True),anomaly_IF],axis=1,sort=False)
df_predict[(df_predict.is_anomaly==-1)]

,time,host,consumer_group,cpu,dbtime,iowait,oth_wait,pga,pxenq,pxqry,...,host_3,host_4,consumer_group_adw_unload_group,consumer_group_dm_tech_group,consumer_group_fp_dm_prior,consumer_group_hdp_unload_group,consumer_group_ods_support_group,consumer_group_sandbox_group,consumer_group_szb_sandbox_group,is_anomaly
22,2018-12-10 17:00:00,1,adw_unload_group,0.803085,0.616750,0.069161,0.366446,0.739366,0.436620,0.181935,...,0,0,1,0,0,0,0,0,0,-1
128,2018-12-10 08:00:00,2,adw_unload_group,0.542601,0.593098,0.608108,0.372493,0.171325,0.809160,0.150862,...,0,0,1,0,0,0,0,0,0,-1
129,2018-12-10 09:00:00,2,adw_unload_group,0.359641,0.403307,0.668919,0.174785,0.134370,0.206107,0.123563,...,0,0,1,0,0,0,0,0,0,-1
178,2018-12-10 00:00:00,2,hdp_unload_group,0.663462,0.464286,0.026733,0.395038,0.018920,0.000000,0.823529,...,0,0,0,0,0,1,0,0,0,-1
274,2018-12-10 10:00:00,3,dm_tech_group,0.733246,0.613146,0.200000,0.463665,0.486204,0.175879,0.074397,...,1,0,0,1,0,0,0,0,0,-1
303,2018-12-10 10:00:00,3,sandbox_group,0.778772,1.000000,1.000000,0.834758,0.309454,0.112821,0.004422,...,1,0,0,0,0,0,0,1,0,-1
381,2018-12-10 01:00:00,4,dm_tech_group,0.799472,0.666310,0.383275,0.496720,0.726645,0.011070,0.648940,...,0,1,0,1,0,0,0,0,0,-1
390,2018-12-10 10:00:00,4,dm_tech_group,0.415567,0.451872,0.254355,0.428304,0.202460,1.000000,0.732948,...,0,1,0,1,0,0,0,0,0,-1


## Autoencoders